In [57]:
from pychord import Chord
import numpy as np
import pygame, pygame.sndarray
import scipy.signal
import requests
import time

from numpy import array
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
import random

In [2]:
#This gets used to translate note names given by pychord into array indices
note_translate = {
    "C2":0,
    "C#2":1,
    "Db2":1,
    "D2":2,
    "D#2":3,
    "Eb2":3,
    "E2":4,
    "F2":5,
    "F#2":6,
    "Gb2":6,
    "G2":7,
    "G#2":8,
    "Ab2":8,
    "A2":9,
    "A#2":10,
    "Bb2":10,
    "B2":11,
    "C3":12,
    "C#3":13,
    "Db3":13,
    "D3":14,
    "D#3":15,
    "Eb3":15,
    "E3":16,
    "F3":17,
    "F#3":18,
    "Gb3":18,
    "G3":19,
    "G#3":20,
    "Ab3":20,
    "A3":21,
    "A#3":22,
    "Bb3":22,
    "B3":23,
    "C4":24,
    "C#4":25,
    "Db4":25,
    "D4":26,
    "D#4":27,
    "Eb4":27,
    "E4":28,
    "F4":29,
    "F#4":30,
    "Gb4":30,
    "G4":31,
    "G#4":32,
    "Ab4":32,
    "A4":33,
    "A#4":34,
    "Bb4":35,
    "B4":36,
}

#This is used to translate note numbers into sine waves
frequencies = """65.41
69.3
73.42
77.78
82.41
87.31
92.5
98
103.83
110
116.54
123.47
130.81
138.59
146.83
155.56
164.81
174.61
185
196
207.65
220
233.08
246.94
261.63
277.18
293.66
311.13
329.63
349.23
369.99
392
415.3
440
466.16
493.88""".split("\n")
frequencies = np.array(frequencies).astype(float)

In [3]:
#Initiate pygame to play music
pygame.mixer.init()
pygame.init()
pygame.mixer.init(frequency=44100, size=-16, channels=1)
sample_rate = 44100
#Window of the lstm (number of chords)
n_steps = 8

In [106]:
#Translate a chord name into an array that can be played
def chord_to_arr(chord):
    blank = np.zeros(36)
    for note in Chord(chord).components_with_pitch(root_pitch=2):
        blank[note_translate[note]] = 1
    return(blank)
#chord_to_arr("Cmaj7")

# def play_for(sample_wave, ms):
#     """Play the given NumPy array, as a sound, for ms milliseconds."""
#     sound = pygame.sndarray.make_sound(sample_wave)
#     sound.play(-1)
#     pygame.time.delay(ms)
#     sound.stop()

def sine_wave(hz, peak, n_samples=sample_rate):
    """Compute N samples of a sine wave with given frequency and peak amplitude.
       Defaults to one second.
    """
    length = sample_rate / float(hz)
    omega = np.pi * 2 / length
    xvalues = np.arange(int(length)) * omega
    onecycle = peak * np.sin(xvalues)
    return np.resize(onecycle, (n_samples,)).astype(np.int16)

#play the notes of a chord based on a name (not used)
def play_chord(chord,short_len=150,long_len=1000):
    size = 44100
    out_sound = sum([sine_wave(440, 0), sine_wave(440, 0)])
    for chord_note in Chord(chord).components_with_pitch(root_pitch=2):
        #2.5 is a pitch multiplier, low frequncy sine waves get rather mushy
        frequency = 2.5*frequencies[note_translate[chord_note]]
        out_sound = sum([out_sound, sine_wave(frequency, 1024)])
        out_sound_reshaped = np.repeat(out_sound.reshape(size, 1), 2, axis = 1)
        sound = pygame.sndarray.make_sound(out_sound_reshaped)
        sound.play()
        pygame.time.wait(int(sound.get_length() * short_len))
        sound.stop()

    out_sound_reshaped = np.repeat(out_sound.reshape(size, 1), 2, axis = 1)
    sound = pygame.sndarray.make_sound(out_sound_reshaped)
    sound.play()
    pygame.time.wait(int(sound.get_length() * long_len))
    sound.stop()
#play_chord("D7")

#play the notes of a chord based on an input array 1 = play this note, 0 = don't play this note
#short_len is the time between notes in the arpeggio
#long_len is the time to hold the final chord
def play_arr(chord_arr,short_len=150,long_len=1000):
    chord_arr = [i for i,v in enumerate(chord_arr) if v > .5]
    size = 44100
    out_sound = sum([sine_wave(440, 0), sine_wave(440, 0)])
    for chord_note in chord_arr:
        frequency = 2.5*frequencies[chord_note]
        out_sound = sum([out_sound, sine_wave(frequency, 1024)])
        out_sound_reshaped = np.repeat(out_sound.reshape(size, 1), 2, axis = 1)
        sound = pygame.sndarray.make_sound(out_sound_reshaped)
        sound.play()
        pygame.time.wait(int(sound.get_length() * short_len))
        sound.stop()

    out_sound_reshaped = np.repeat(out_sound.reshape(size, 1), 2, axis = 1)
    sound = pygame.sndarray.make_sound(out_sound_reshaped)
    sound.play()
    pygame.time.wait(int(sound.get_length() * long_len))
    sound.stop()
#play_arr([1., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 1., 0., 0., 1., 0., 0.,
#       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
#       0., 0.])



#Convert the long chord array into many arrays of length 8
def split_sequence(sequence, n_steps):
    X, y = list(), list()
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the sequence
        if end_ix > len(sequence)-1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)

#Convert fuzzy predictions to binary notes (chord array) based on a tolerance
#Also shifts the previous input left 1 and appends the new chord array to the input
def generate_input(x_input,song_len, tolerance = 0.5, tolerance_var=0):
    output_arr = []
    x_input = x_input.reshape((1, n_steps, n_features))
    for i in range(song_len):
        yhat = model.predict(x_input, verbose=0)
        yhat = np.where(yhat >= tolerance+random.uniform(-1*tolerance_var, tolerance_var), 1, 0)
        output_arr.append(yhat[0])
        x_input = np.roll(x_input, -1,axis=1)
        x_input[[-1]] = yhat
    return(np.array(output_arr))

#Convert fuzzy predictions to binary notes (chord array) based on the top 4 notes
#Also shifts the previous input left 1 and appends the new chord array to the input
def generate_input_notes(x_input,song_len, notes = 4):
    output_arr = []
    x_input = x_input.reshape((1, n_steps, n_features))
    for i in range(song_len):
        yhat = model.predict(x_input, verbose=0)
        chord_notes = np.argsort(yhat)[::-1]
        yhat = np.zeros(36)
        for note in chord_notes[0][0:notes]:
            yhat[note] = 1
        output_arr.append(yhat)
        x_input = np.roll(x_input, -1,axis=1)
        x_input[[-1]] = yhat
    return(np.array(output_arr))

In [110]:
#put all the gathered chords into one array
chord_arr = []

#This scraping technique can grab all the songs on the first page of an artist on e-chords
#It can easily be modified to look by genre, name, or browse multiple pages
URL = "https://www.e-chords.com/nat-king-cole"
page = requests.get(URL)
url_list = page.text.split("<p class=\"itm h2l\"><a href=\"")[1:]
for i in range(len(url_list)):
    url_list[i]=url_list[i].split("\">")[0]
    
print("Number of songs on page: "+str(len(url_list)))

#### THIS IS HOW TO GET YOUR IP BLOCKED FOR BREAKING THE TERMS OF SERVICE
#### OF THE CHORD WEBSITE YOU USE
#### USE A VPN AND ACCEPT THE RISK AT YOUR OWN DISCRETION
input("Don't run this without a vpn on, and even then be cautious. You can get your IP blocked for webscraping. "+\
     "You can adjust the wait time between pages with time.sleep(wait_time). Longer times are safer but slower.")
#Scrape the (first 100) songs
for URL in url_list[0:100]:
    time.sleep(1)
    try:
        page = requests.get(URL)
        #pad with appropriate number of zeros (this can probably be removed)
        for _i in range(n_steps):
            chord_arr.append(np.zeros(36))
        chords = page.text.split("<u>")[1:]
        for i in range(len(chords)):
            chords[i]=chords[i].split("</u>")[0]
        for chord in chords:
            try:
                chord_arr.append(chord_to_arr(chord))
            except:
                pass
    except:
        pass
    
print("Number of chords: "+str(len(chord_arr)))

#play a short sample from the chord list
for chord in chord_arr[10:15]:
    play_arr(chord)

Number of songs on page: 249
Don't run this without a vpn on, and even then be cautious. You can get your IP blocked for webscraping. You can adjust the wait time between pages with time.sleep(wait_time). Longer times are safer but slower.
Number of chords: 110


In [47]:
#URL = "https://www.e-chords.com/chords/ella-fitzgerald/between-the-devil-and-the-deep-blue-sea"
#page = requests.get(URL)

### To play the chords of a song
# chords = page.text.split("<u>")[1:]
# for i in range(len(chords)):
#     chords[i]=chords[i].split("</u>")[0]
# for chord in chords[0:10]:
#     time.sleep(.1)
#     try:
#         play_chord(chord)
#     except:
#         print(chord)
#         pass



In [90]:
#Train an lstm on the scraped music
print("Training...")
start_time = time.time()
# number of time steps
#continue training from old model?
resume=False
n_epochs = 2000
# split into samples
X, y = split_sequence(chord_arr, n_steps)
# reshape from [samples, timesteps] into [samples, timesteps, features]
n_features = len(chord_arr[0])
X = X.reshape((X.shape[0], X.shape[1], n_features))
# define model
if not resume:
    model = Sequential()
    model.add(LSTM(50, activation='relu', input_shape=(n_steps, n_features)))
    model.add(Dense(n_features))
    model.compile(optimizer='adam', loss='mse')
# fit model
model.fit(X, y, epochs=n_epochs, verbose=0)
print("Done!  Took "+str(int(time.time() - start_time))+" seconds")

Training...
Done!  Took 1616 seconds


In [115]:
#This version picks any number of notes based on a threshold/tolerance, 
#with some randomness to the threshold to give variety
#Start with all notes on to get it kicked off
# input_chord_arr = []
# for _i in range(n_steps):
#     input_chord_arr.append(np.ones(36))
# input_chord_arr = np.array(input_chord_arr)

# gen_chord_arr = (generate_input(input_chord_arr, song_len=40, tolerance=.5, tolerance_var=0.1))
# for chord in gen_chord_arr:
#     play_arr(chord,400,300)

#This version picks the top 4 notes and sounds better IMO
#Start out with all notes off
input_chord_arr = []
for _i in range(n_steps):
    input_chord_arr.append(np.zeros(36))
input_chord_arr = np.array(input_chord_arr)
#Play the song!
gen_chord_arr = (generate_input_notes(input_chord_arr, song_len=40))
for chord in gen_chord_arr:
    play_arr(chord,400,600)

KeyboardInterrupt: 